In [ ]:
minimum_gesture_samples = 300

maximum_gesture_samples = 600

HISTERESE = 1.6

THRESHOLD_IN = 1.7

THRESHOLD_OUT = 2

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from statistics import mean

calc_mean_start = 50

n_samples = []

for subject_number in range(5):
    for gesture_number in range(1,6):
        for file_number in range(10):
            
            file_sampled  = "C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_"+str(subject_number)+"/G"+str(gesture_number)+"/Emg_"+str(file_number)+".csv"
            file_segmented  = "C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_segmented/s_"+str(subject_number)+"/G"+str(gesture_number) +"/Emg_"+str(file_number)+".csv"

            df = pd.read_csv(file_sampled)

            for j in df.drop(['timestamp'],axis=1).columns:
                filtered_signal = df[j].to_list()
                filtered_signal_2 = filtered_signal.copy()
                for i in range(2,len(df)):
                    if filtered_signal[i] != 0 :
                        filtered_signal[i] = filtered_signal_2[i-1]**2 -filtered_signal_2[i]*filtered_signal_2[i-2]
                df[j + '_filtered'] = filtered_signal


            for j in range(0,8):
                df['EMG_s'+str(j)+'_media_movel'] = df['EMG_s'+str(j)+'_filtered'].rolling(100).mean()

            df['total_nrg'] = pd.Series(0, index=np.arange(len(df)))
            for j in range(0,8):
                df.total_nrg        = df.total_nrg        + df['EMG_s'+str(j)+'_media_movel'].apply(lambda x : abs(x))

            calculated_mean = mean(df.total_nrg[df.total_nrg.first_valid_index():].head(calc_mean_start).dropna())

            df['signal'] = 0
            
            signal_temp = [0] * len(df)

            gesture_count = 0

            sample_counter = 0

            is_signal = False

            for i in range(calc_mean_start,len(df)):
                # print(calculated_mean)
                if is_signal:
                    if ((df.total_nrg[i] > THRESHOLD_OUT *calculated_mean) or sample_counter<minimum_gesture_samples) and sample_counter<maximum_gesture_samples:
                        signal_temp[i] = gesture_count
                    else:
                        is_signal = False
                    sample_counter = sample_counter + 1
                else:  
                    if df.total_nrg[i] > THRESHOLD_IN * calculated_mean:
                        gesture_count = gesture_count + 1 
                        signal_temp[i] = gesture_count
                        is_signal = True
                    else:
                        if df.total_nrg[i] < HISTERESE* calculated_mean:
                            calculated_mean = (calculated_mean*(i-1)+df.total_nrg[i]*5)/(i+4)
                    sample_counter = 0


            df['signal'] = signal_temp

            df = df.drop(['EMG_s0_filtered', 'EMG_s1_filtered', 'EMG_s2_filtered', 'EMG_s3_filtered', 'EMG_s4_filtered', 'EMG_s5_filtered', 'EMG_s6_filtered', 'EMG_s7_filtered', 'EMG_s0_media_movel', 'EMG_s1_media_movel', 'EMG_s2_media_movel', 'EMG_s3_media_movel', 'EMG_s4_media_movel', 'EMG_s5_media_movel', 'EMG_s6_media_movel', 'EMG_s7_media_movel'], axis = 1)

            df.to_csv(file_segmented)
            print('Processado: ' + file_sampled + " : " +  str(df['signal'].max()))
            n_samples.append(df['signal'].max())

n_samples = np.array(n_samples)

parameters_dict = {'HISTERESE': HISTERESE,
                   'THRESHOLD_IN': THRESHOLD_IN,
                   'THRESHOLD_OUT': THRESHOLD_OUT,
                   'n_samples': n_samples,
                  }

parameters = pd.DataFrame(data= parameters_dict)

parameters.to_excel('parameters.xlsx')

Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_0/G1/Emg_0.csv : 13
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_0/G1/Emg_1.csv : 14
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_0/G1/Emg_2.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_0/G1/Emg_3.csv : 14
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_0/G1/Emg_4.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_0/G1/Emg_5.csv : 13
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_0/G1/Emg_6.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_0/G1/Emg_7.csv : 14
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_0/G1/Emg_8.csv : 15
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_0/G1/Emg_9.csv : 13
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_0/G2/Emg_0.csv : 1
Processado: C:/Users/Bioteca/Desk

Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_1/G5/Emg_4.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_1/G5/Emg_5.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_1/G5/Emg_6.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_1/G5/Emg_7.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_1/G5/Emg_8.csv : 10
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_1/G5/Emg_9.csv : 12
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_2/G1/Emg_0.csv : 10
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_2/G1/Emg_1.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_2/G1/Emg_2.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_2/G1/Emg_3.csv : 13
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_2/G1/Emg_4.csv : 11
Processado: C:/Users/Bioteca/Des

Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_3/G4/Emg_8.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_3/G4/Emg_9.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_3/G5/Emg_0.csv : 12
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_3/G5/Emg_1.csv : 10
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_3/G5/Emg_2.csv : 13
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_3/G5/Emg_3.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_3/G5/Emg_4.csv : 12
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_3/G5/Emg_5.csv : 11
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_3/G5/Emg_6.csv : 14
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_3/G5/Emg_7.csv : 14
Processado: C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_3/G5/Emg_8.csv : 15
Processado: C:/Users/Bioteca/Des

In [ ]:
calc_mean_start = 50

for subject_number in range(6):
    
    for file_number in range(5):

        file_sampled  = "D:/matc/Documents/Projects/Subjects_sampled/s_"+str(subject_number)+"/Sequences/Emg_"+str(file_number)+".csv"
        file_segmented  = "D:/matc/Documents/Projects/Subjects_segmented/s_"+str(subject_number)+"/Sequences/Emg_"+str(file_number)+".csv"

        df = pd.read_csv(file_sampled)

        for j in df.drop(['timestamp'],axis=1).columns:
            filtered_signal = df[j].to_list()
            filtered_signal_2 = filtered_signal.copy()
            for i in range(2,len(df)):
                if filtered_signal[i] != 0 :
                    filtered_signal[i] = filtered_signal_2[i-1]**2 -filtered_signal_2[i]*filtered_signal_2[i-2]
            df[j + '_filtered'] = filtered_signal

        for j in range(0,8):
            df['EMG_s'+str(j)+'_media_movel'] = df['EMG_s'+str(j)+'_filtered'].rolling(100).mean()

        df.total_nrg = pd.Series(0, index=np.arange(len(df)))
        for j in range(0,8):
            df.total_nrg        = df.total_nrg        + df['EMG_s'+str(j)+'_media_movel'].apply(lambda x : abs(x))

        calculated_mean = mean(df.total_nrg[df.total_nrg.first_valid_index():].head(calc_mean_start).dropna())

        df['signal'] = 0

        signal_temp = [0] * len(df)
        
        HISTERESE = 1.6
        
        THRESHOLD_IN = 2
        
        THRESHOLD_OUT = 1.7

        gesture_count = 0

        minimum_gesture_samples = 300

        maximum_gesture_samples = 600

        sample_counter = 0

        is_signal = False

        for i in range(calc_mean_start,len(df)):
            # print(calculated_mean)
            if is_signal:
                if ((df.total_nrg[i] > THRESHOLD_OUT *calculated_mean) or sample_counter<minimum_gesture_samples) and sample_counter<maximum_gesture_samples:
                    signal_temp[i] = gesture_count
                else:
                    is_signal = False
                sample_counter = sample_counter + 1
            else:  
                if df.total_nrg[i] > THRESHOLD_IN * calculated_mean:
                    gesture_count = gesture_count + 1 
                    signal_temp[i] = gesture_count
                    is_signal = True
                else:
                    if df.total_nrg[i] < HISTERESE * calculated_mean:
                        calculated_mean = (calculated_mean*(i-1)+df.total_nrg[i]*5)/(i+4)
                sample_counter = 0


        df['signal'] = signal_temp

        df = df.drop(['EMG_s0_filtered', 'EMG_s1_filtered', 'EMG_s2_filtered', 'EMG_s3_filtered', 'EMG_s4_filtered', 'EMG_s5_filtered', 'EMG_s6_filtered', 'EMG_s7_filtered', 'EMG_s0_media_movel', 'EMG_s1_media_movel', 'EMG_s2_media_movel', 'EMG_s3_media_movel', 'EMG_s4_media_movel', 'EMG_s5_media_movel', 'EMG_s6_media_movel', 'EMG_s7_media_movel'], axis = 1)

        df.to_csv(file_segmented)
        print('Processado: ' + file_sampled + " : " +  str(df['signal'].max()))

In [ ]:
calc_mean_start = 50


for gesture_number in range(1,6):

    file_sampled  = "D:/matc/Documents/Projects/Subjects_sampled/s_5/G"+str(gesture_number)+"/Emg_0.csv"
    file_segmented  = "D:/matc/Documents/Projects/Subjects_segmented/s_5/G"+str(gesture_number) +"/Emg_0.csv"

    df = pd.read_csv(file_sampled)

    for j in df.drop(['timestamp'],axis=1).columns:
        filtered_signal = df[j].to_list()
        filtered_signal_2 = filtered_signal.copy()
        for i in range(2,len(df)):
            if filtered_signal[i] != 0 :
                filtered_signal[i] = filtered_signal_2[i-1]**2 -filtered_signal_2[i]*filtered_signal_2[i-2]
        df[j + '_filtered'] = filtered_signal


    for j in range(0,8):
        df['EMG_s'+str(j)+'_media_movel'] = df['EMG_s'+str(j)+'_filtered'].rolling(100).mean()

    df.total_nrg = pd.Series(0, index=np.arange(len(df)))
    for j in range(0,8):
        df.total_nrg        = df.total_nrg        + df['EMG_s'+str(j)+'_media_movel'].apply(lambda x : abs(x))

    calculated_mean = mean(df.total_nrg[df.total_nrg.first_valid_index():].head(calc_mean_start).dropna())

    df['signal'] = 0

    signal_temp = [0] * len(df)

    gesture_count = 0

    minimum_gesture_samples = 300
    
    maximum_gesture_samples = 600
    
    sample_counter = 0

    is_signal = False
    
    HISTERESE = 1.6

    THRESHOLD_IN = 2

    THRESHOLD_OUT = 1.7

    for i in range(calc_mean_start,len(df)):
        # print(calculated_mean)
        if is_signal:
            if ((df.total_nrg[i] > THRESHOLD_OUT *calculated_mean) or sample_counter<minimum_gesture_samples) and sample_counter<maximum_gesture_samples:
                signal_temp[i] = gesture_count
            else:
                is_signal = False
            sample_counter = sample_counter + 1
        else:  
            if df.total_nrg[i] > THRESHOLD_IN * calculated_mean:
                gesture_count = gesture_count + 1 
                signal_temp[i] = gesture_count
                is_signal = True
            else:
                if df.total_nrg[i] < HISTERESE * calculated_mean:
                    calculated_mean = (calculated_mean*(i-1)+df.total_nrg[i]*5)/(i+4)
            sample_counter = 0

    df['signal'] = signal_temp

    df = df.drop(['EMG_s0_filtered', 'EMG_s1_filtered', 'EMG_s2_filtered', 'EMG_s3_filtered', 'EMG_s4_filtered', 'EMG_s5_filtered', 'EMG_s6_filtered', 'EMG_s7_filtered', 'EMG_s0_media_movel', 'EMG_s1_media_movel', 'EMG_s2_media_movel', 'EMG_s3_media_movel', 'EMG_s4_media_movel', 'EMG_s5_media_movel', 'EMG_s6_media_movel', 'EMG_s7_media_movel'], axis = 1)

    df.to_csv(file_segmented)
    print('Processado: ' + file_sampled + " : " +  str(df['signal'].max()))